In [13]:
from data import *
import pickle
from architectures.datasets import *

c:\Users\roman\miniforge3\envs\AI\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
path_data = "D:\\OneDrive\\OneDrive - enpc.fr\\Roman Cloud\\altegrad_challenge_2022\\"
adj = pickle.load(open(path_data + 'adj.pkl', 'rb'))
features = pickle.load(open(path_data + 'features.pkl', 'rb'))
edge_features = pickle.load(open(path_data + 'edge_features.pkl', 'rb'))

In [8]:
adj_train, features_train, edge_features_train, y_train, adj_test, features_test,\
        edge_features_test, proteins_test = split_train_test(adj, features, edge_features, path=path_data)

In [39]:
class DGLGraphDataset(Dataset):
    def __init__(self, adj, features, edge_features, labels=None, train=True):
        self.adj = adj
        self.features = features
        self.edge_features = edge_features
        self.labels = labels
        self.train = train

    def __len__(self):
        return len(self.adj)

    def __getitem__(self, idx):
        g = dgl.from_scipy(self.adj[idx])
        g.ndata['feat'] = torch.FloatTensor(self.features[idx])
        g.edata['feat'] = torch.FloatTensor(self.edge_features[idx])
        
        if self.train :
            return g, self.labels[idx]
        else : 
            return g

In [40]:
dataset_train = DGLGraphDataset(adj_train, features_train, edge_features_train, y_train)


In [41]:
g = dataset_train[0][0]

In [43]:
def edge_filter(edata):
    return edata['feat'][0] == 1

g_sub = g.filter_edges(edge_filter)

TypeError: edge_filter() missing 1 required positional argument: 'eid'

In [ ]:
def has_dst_one(edges): return (edges.dst['x'] == 1).squeeze(1)

In [23]:
import dgl

# create a graph
g = dgl.DGLGraph()
g.add_nodes(4)
g.add_edges([0, 1, 2, 3], [1, 2, 3, 0])

# add edge features
edge_features = torch.Tensor([[1, 2, 3], [0, 1, 2], [1, 0, 0], [0, 1, 1]])
g.edata['feat'] = edge_features

# filter edges
subg = g.filter_edges(lambda edges: edges.data['feat'][:, 0] == 1)

c:\Users\roman\miniforge3\envs\AI\lib\site-packages\dgl\heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'
